In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

# 定義CNN模型
def create_cnn_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax')) # 10個類別的輸出層
    return model

# 載入資料集
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 預處理資料
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

# 創建模型
model = create_cnn_model()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 訓練模型
model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.2)

# 評估模型
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

Epoch 1/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.8526 - loss: 0.4705 - val_accuracy: 0.9797 - val_loss: 0.0648
Epoch 2/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9837 - loss: 0.0510 - val_accuracy: 0.9843 - val_loss: 0.0559
Epoch 3/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9884 - loss: 0.0381 - val_accuracy: 0.9875 - val_loss: 0.0447
Epoch 4/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9925 - loss: 0.0245 - val_accuracy: 0.9882 - val_loss: 0.0405
Epoch 5/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.9937 - loss: 0.0204 - val_accuracy: 0.9875 - val_loss: 0.0399
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9869 - loss: 0.0393
Test accuracy: 0.9897000193595886


Store Images In Sql

In [3]:
import csv
import json
import mysql.connector
import pandas as pd
import sqlalchemy as sql
from sqlalchemy import text
import numpy as np


with open('final_config.json') as f:
    config = json.load(f)
host = config['host']
user = config['user']
passwd = config['passwd']

In [4]:
cnx = mysql.connector.connect(
    host=host,
    user=user,
    password=passwd,
    database='my_database'
)

In [6]:
# Create a cursor object
cur = cnx.cursor()

# Create a table for storing images
# id SERIAL PRIMARY KEY,
create_table_query = """
CREATE TABLE images (
    id INT AUTO_INCREMENT PRIMARY KEY,
    data text
);
"""
cur.execute(create_table_query)
cnx.commit()

In [7]:

# Convert the NumPy array to a list and then to a JSON string
for image_data in train_images:
    # Convert the NumPy array to a list and then to a JSON string
    train_images_list = image_data.tolist()
    train_images_json = json.dumps(train_images_list)
    
    # Insert the JSON string into the database
    cur.execute("INSERT INTO images (data) VALUES (%s)",(train_images_json,))
    cnx.commit()


# Close the cursor and connection
cur.close()
cnx.close()